In [1]:
# the objective of this notebook is to collect data from 2 APIs and store them as csv Files 
# The first API will be the Universalis API to collect 
# Historical market data in the consumable market in FFXIV
# The second API will be the XIVAPI to collect
# Item data and World data for the consumable market in FFXIV
# The data will be stored in csv files for further analysis

# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Universalis API no key required
# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'


In [2]:
#test cell to see if the API is working

import urllib.request, json

request = urllib.request.Request("http://xivapi.com/Item/1675")
request.add_header('User-Agent', '&lt;User-Agent&gt;')
data = json.loads(urllib.request.urlopen(request).read())

# I.Lv 80 Curtana
print ("I.Lv %d %s" % (data['LevelItem'], data['Name_en']))

I.Lv 80 Curtana


In [3]:
# lets start by getting the list of all the consumables in the game
# Consumables are items that can be used to provide a benefit to the player
# This includes food, potions, and other items that can be used to provide a
# temporary buff to the player
# The list of consumables will be used to collect the historical market data
# for each of the consumables
# The list of consumables will be collected from the XIVAPI
# The XIVAPI has a rate limit of 30 requests per second
# The category ID for consumables is 46
# The category ID for medicines is 44
# I can use the ItemKind.ID=5 parameter to get all items that are "Medicines & Meals" 

In [4]:
import urllib.request
import json
import pandas as pd

# Initialize an empty list to store all results
all_results = []

# Initialize page to 1
page = 1

while True:
    # Make the request
    request = urllib.request.Request(f"https://xivapi.com/search?filters=ItemKind.ID=5&limit=250&page={page}&private_key={xivapi_key}")
    request.add_header('User-Agent', '&lt;User-Agent&gt;')
    data = json.loads(urllib.request.urlopen(request).read())
    # The data is a dictionary with a key of "Results" and a value of a list of dictionaries

    # Add the results to our list
    all_results.extend(data['Results'])
    # The list of dictionaries contains the data for each of the consumables
    # The data for each consumable includes the name, the ID, and the category

    # Check if there are more pages
    if 'Pagination' in data and data['Pagination']['PageNext']:
        # If there are, increment the page counter
        page += 1
    else:
        # If there aren't, break the loop
        break

# Create a dataframe with all the results
consumables_df = pd.DataFrame(all_results)

# Display the first few rows of the dataframe
print(consumables_df)

# Save the data to a csv file
consumables_df.to_csv('..\\data\\consumables.csv', index=False)

         ID                  Icon                  Name          Url UrlType  \
0      4551  /i/020000/020601.png                Potion   /Item/4551    Item   
1      4552  /i/020000/020602.png             Hi-Potion   /Item/4552    Item   
2      4553  /i/020000/020603.png           Mega-Potion   /Item/4553    Item   
3      4554  /i/020000/020603.png              X-Potion   /Item/4554    Item   
4      4555  /i/020000/020621.png                 Ether   /Item/4555    Item   
...     ...                   ...                   ...          ...     ...   
2703  41411  /i/029000/029780.png     Lopoceras Elegans  /Item/41411    Item   
2704  41412  /i/028000/028969.png        Sidereal Whale  /Item/41412    Item   
2705  41413  /i/025000/025314.png    Prime Kukuru Beans  /Item/41413    Item   
2706  41414  /i/025000/025329.png  Sublime Kukuru Beans  /Item/41414    Item   
2707  41419  /i/029000/029586.png      Stargilt Lobster  /Item/41419    Item   

         _  _Score  
0     item       0

In [5]:
# Now that we have the list of consumables, we can use the Universalis API to collect the historical market data for each of the consumables
# The Universalis API has a rate limit of 25 requests per second
# The Universalis API provides historical market data for each of the servers in the game
# The historical market data includes the price and the quantity of the item

# First we need a nlist of all the servers in the game
# This will include Data Centers and Worlds
# The Data Centers are the larger groupings of servers based on location/region
# The Worlds are the individual servers that players can play on
# Because the Data Centers support cross-server play, we will group the data by Data Center

# https://universalis.app/api/v2/data-centers will return a list of all the data centers

data_centers = requests.get('https://universalis.app/api/v2/data-centers')


data_centers

<Response [200]>

In [6]:
data_centers = data_centers.json()
data_centers

[{'name': 'Elemental',
  'region': 'Japan',
  'worlds': [45, 49, 50, 58, 68, 72, 90, 94]},
 {'name': 'Gaia',
  'region': 'Japan',
  'worlds': [43, 46, 51, 59, 69, 76, 92, 98]},
 {'name': 'Mana',
  'region': 'Japan',
  'worlds': [23, 28, 44, 47, 48, 61, 70, 96]},
 {'name': 'Aether',
  'region': 'North-America',
  'worlds': [40, 54, 57, 63, 65, 73, 79, 99]},
 {'name': 'Primal',
  'region': 'North-America',
  'worlds': [35, 53, 55, 64, 77, 78, 93, 95]},
 {'name': 'Chaos',
  'region': 'Europe',
  'worlds': [39, 71, 80, 83, 85, 97, 400, 401]},
 {'name': 'Light',
  'region': 'Europe',
  'worlds': [33, 36, 42, 56, 66, 67, 402, 403]},
 {'name': 'Crystal',
  'region': 'North-America',
  'worlds': [34, 37, 41, 62, 74, 75, 81, 91]},
 {'name': 'Materia', 'region': 'Oceania', 'worlds': [21, 22, 86, 87, 88]},
 {'name': 'Meteor',
  'region': 'Japan',
  'worlds': [24, 29, 30, 31, 32, 52, 60, 82]},
 {'name': 'Dynamis',
  'region': 'North-America',
  'worlds': [404, 405, 406, 407]},
 {'name': 'NA Cloud 

In [7]:
#normalize the data
data_centers = pd.json_normalize(data_centers)
data_centers

,name,region,worlds
0,Elemental,Japan,"[45, 49, 50, 58, 68, 72, 90, 94]"
1,Gaia,Japan,"[43, 46, 51, 59, 69, 76, 92, 98]"
2,Mana,Japan,"[23, 28, 44, 47, 48, 61, 70, 96]"
3,Aether,North-America,"[40, 54, 57, 63, 65, 73, 79, 99]"
4,Primal,North-America,"[35, 53, 55, 64, 77, 78, 93, 95]"
5,Chaos,Europe,"[39, 71, 80, 83, 85, 97, 400, 401]"
6,Light,Europe,"[33, 36, 42, 56, 66, 67, 402, 403]"
7,Crystal,North-America,"[34, 37, 41, 62, 74, 75, 81, 91]"
8,Materia,Oceania,"[21, 22, 86, 87, 88]"
9,Meteor,Japan,"[24, 29, 30, 31, 32, 52, 60, 82]"


In [8]:
# Save the data to a csv file
data_centers.to_csv('..\\data\\data_centers.csv', index=False)

In [9]:
# Now to use https://universalis.app/api/v2/worlds to get a list of all the worlds
# This will return a list of all the worlds in the game
# This will include the World ID, and the World Name

worlds = requests.get('https://universalis.app/api/v2/worlds')
worlds

<Response [200]>

In [10]:
worlds = worlds.json()
worlds  

[{'id': 21, 'name': 'Ravana'},
 {'id': 22, 'name': 'Bismarck'},
 {'id': 23, 'name': 'Asura'},
 {'id': 24, 'name': 'Belias'},
 {'id': 28, 'name': 'Pandaemonium'},
 {'id': 29, 'name': 'Shinryu'},
 {'id': 30, 'name': 'Unicorn'},
 {'id': 31, 'name': 'Yojimbo'},
 {'id': 32, 'name': 'Zeromus'},
 {'id': 33, 'name': 'Twintania'},
 {'id': 34, 'name': 'Brynhildr'},
 {'id': 35, 'name': 'Famfrit'},
 {'id': 36, 'name': 'Lich'},
 {'id': 37, 'name': 'Mateus'},
 {'id': 39, 'name': 'Omega'},
 {'id': 40, 'name': 'Jenova'},
 {'id': 41, 'name': 'Zalera'},
 {'id': 42, 'name': 'Zodiark'},
 {'id': 43, 'name': 'Alexander'},
 {'id': 44, 'name': 'Anima'},
 {'id': 45, 'name': 'Carbuncle'},
 {'id': 46, 'name': 'Fenrir'},
 {'id': 47, 'name': 'Hades'},
 {'id': 48, 'name': 'Ixion'},
 {'id': 49, 'name': 'Kujata'},
 {'id': 50, 'name': 'Typhon'},
 {'id': 51, 'name': 'Ultima'},
 {'id': 52, 'name': 'Valefor'},
 {'id': 53, 'name': 'Exodus'},
 {'id': 54, 'name': 'Faerie'},
 {'id': 55, 'name': 'Lamia'},
 {'id': 56, 'name': 

In [11]:
#normalize the data
worlds = pd.json_normalize(worlds)
worlds

,id,name
0,21,Ravana
1,22,Bismarck
2,23,Asura
3,24,Belias
4,28,Pandaemonium
...,...,...
114,2075,카벙클
115,2076,초코보
116,2077,모그리
117,2078,톤베리


In [12]:
# Save the data to a csv file
worlds.to_csv('..\\data\\worlds.csv', index=False)

In [13]:
# Next we will get a list of all marketable items in the game https://universalis.app/api/v2/marketable
# This will return a list of all the itemIDs that are marketable

marketable = requests.get('https://universalis.app/api/v2/marketable')
marketable


<Response [200]>

In [14]:
marketable = marketable.json()
marketable

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 1601,
 1602,
 1603,
 1604,
 1605,
 1606,
 1607,
 1609,
 1611,
 1613,
 1614,
 1616,
 1621,
 1622,
 1625,
 1627,
 1633,
 1635,
 1636,
 1637,
 1639,
 1642,
 1643,
 1648,
 1649,
 1650,
 1657,
 1659,
 1662,
 1663,
 1666,
 1670,
 1673,
 1680,
 1681,
 1682,
 1683,
 1684,
 1685,
 1686,
 1688,
 1694,
 1697,
 1699,
 1701,
 1706,
 1708,
 1711,
 1716,
 1723,
 1728,
 1731,
 1732,
 1733,
 1736,
 1740,
 1743,
 1749,
 1750,
 1751,
 1752,
 1753,
 1754,
 1756,
 1758,
 1764,
 1766,
 1769,
 1771,
 1776,
 1778,
 1781,
 1786,
 1796,
 1799,
 1800,
 1801,
 1803,
 1806,
 1810,
 1813,
 1819,
 1820,
 1821,
 1822,
 1823,
 1824,
 1825,
 1827,
 1833,
 1836,
 1838,
 1840,
 1845,
 1847,
 1850,
 1855,
 1862,
 1867,
 1870,
 1871,
 1872,
 1875,
 1879,
 1882,
 1889,
 1891,
 1892,
 1893,
 1894,
 1895,
 1897,
 1899,
 1905,
 1908,
 1910,
 1915,
 1917,
 1920,
 1925,
 1932,
 1937,
 1940,
 1941,
 1942,
 1945,
 1949,
 1953,
 1958,
 1959,
 1960,
 

In [15]:
# no need to normalize this data as it is a list of itemIDs
marketable_df = pd.DataFrame(marketable)
marketable_df


,0
0,2
1,3
2,4
3,5
4,6
...,...
14532,41662
14533,41663
14534,41664
14535,41665


In [16]:
# rename the 0 column to itemID
marketable_df.rename(columns={0: 'ID'}, inplace=True)
marketable_df

,ID
0,2
1,3
2,4
3,5
4,6
...,...
14532,41662
14533,41663
14534,41664
14535,41665


In [17]:
# save the data to a csv file
# with this list of marketable items we can filter our consumables list to only include items that are marketable
marketable_df.to_csv('..\\data\\marketable.csv', index=False)

In [18]:
consumables_df

,ID,Icon,Name,Url,UrlType,_,_Score
0,4551,/i/020000/020601.png,Potion,/Item/4551,Item,item,0
1,4552,/i/020000/020602.png,Hi-Potion,/Item/4552,Item,item,0
2,4553,/i/020000/020603.png,Mega-Potion,/Item/4553,Item,item,0
3,4554,/i/020000/020603.png,X-Potion,/Item/4554,Item,item,0
4,4555,/i/020000/020621.png,Ether,/Item/4555,Item,item,0
...,...,...,...,...,...,...,...
2703,41411,/i/029000/029780.png,Lopoceras Elegans,/Item/41411,Item,item,0
2704,41412,/i/028000/028969.png,Sidereal Whale,/Item/41412,Item,item,0
2705,41413,/i/025000/025314.png,Prime Kukuru Beans,/Item/41413,Item,item,0
2706,41414,/i/025000/025329.png,Sublime Kukuru Beans,/Item/41414,Item,item,0


In [19]:
# Now that we have the list of marketable items, we can filter our consumables list to only include items that are marketable

# Merge the consumables list with the marketable list
marketable_consumables_df = consumables_df.merge(marketable_df, left_on='ID', right_on='ID')
marketable_consumables_df

,ID,Icon,Name,Url,UrlType,_,_Score
0,4551,/i/020000/020601.png,Potion,/Item/4551,Item,item,0
1,4552,/i/020000/020602.png,Hi-Potion,/Item/4552,Item,item,0
2,4553,/i/020000/020603.png,Mega-Potion,/Item/4553,Item,item,0
3,4554,/i/020000/020603.png,X-Potion,/Item/4554,Item,item,0
4,4555,/i/020000/020621.png,Ether,/Item/4555,Item,item,0
...,...,...,...,...,...,...,...
2316,41408,/i/028000/028116.png,Gharlichthys,/Item/41408,Item,item,0
2317,41409,/i/028000/028562.png,Snowy Parexus,/Item/41409,Item,item,0
2318,41410,/i/028000/028564.png,Furcacauda,/Item/41410,Item,item,0
2319,41411,/i/029000/029780.png,Lopoceras Elegans,/Item/41411,Item,item,0


In [20]:
# Save the data to a csv file
marketable_consumables_df.to_csv('..\\data\\marketable_consumables.csv', index=False)

In [21]:
# Now that we have the list of marketable consumables, we can use the Universalis API to collect the historical market data for each of the consumables
# The Universalis API has a rate limit of 25 requests per second
# https://universalis.app/api/v2/history/{WorldID}/{itemIDs}?entriesToReturn=999999&minSalePrice=0&maxSalePrice=2147483647

# Test a single request to the Universalis API
# The API accepts strings as inputs for either Worlds of Data centers
# The item ID for the Grade 8 Tincture of Vitality is 39729

# Make the request
world = "Aether" # data center made up of 8 worlds
itemID = 27759
histInSeconds = 4*365*24*60*60 # 4 year in seconds

response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
response



<Response [200]>

In [22]:
response = response.json()
response

{'itemID': 27759,
 'lastUploadTime': 1710809929396,
 'entries': [{'hq': False,
   'pricePerUnit': 200,
   'quantity': 72,
   'buyerName': 'Pomo Pamo',
   'onMannequin': False,
   'timestamp': 1710808553,
   'worldName': 'Cactuar',
   'worldID': 79},
  {'hq': False,
   'pricePerUnit': 187,
   'quantity': 99,
   'buyerName': 'Venase Bosco',
   'onMannequin': False,
   'timestamp': 1710808314,
   'worldName': 'Cactuar',
   'worldID': 79},
  {'hq': False,
   'pricePerUnit': 205,
   'quantity': 99,
   'buyerName': 'Arzedeu Nebulus',
   'onMannequin': False,
   'timestamp': 1710808074,
   'worldName': 'Midgardsormr',
   'worldID': 65},
  {'hq': False,
   'pricePerUnit': 187,
   'quantity': 99,
   'buyerName': 'Wren Evermist',
   'onMannequin': False,
   'timestamp': 1710806348,
   'worldName': 'Cactuar',
   'worldID': 79},
  {'hq': False,
   'pricePerUnit': 200,
   'quantity': 10,
   'buyerName': 'Celestial Tigress',
   'onMannequin': False,
   'timestamp': 1710788946,
   'worldName': 'Midga

In [23]:
#normalize the data
response = pd.json_normalize(response)
response

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.90,stackSizeHistogramNQ.91,stackSizeHistogramNQ.92,stackSizeHistogramNQ.93,stackSizeHistogramNQ.94,stackSizeHistogramNQ.95,stackSizeHistogramNQ.96,stackSizeHistogramNQ.97,stackSizeHistogramNQ.98,stackSizeHistogramNQ.99
0,27759,1710809929396,"[{'hq': False, 'pricePerUnit': 200, 'quantity'...",Aether,2785.2856,2785.2856,0,1769,1909,1645,...,131,51,56,57,56,75,68,45,63,12248


In [24]:
#normalize the entries column
entries = pd.json_normalize(response['entries'][0])
entries

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,False,200,72,Pomo Pamo,False,1710808553,Cactuar,79
1,False,187,99,Venase Bosco,False,1710808314,Cactuar,79
2,False,205,99,Arzedeu Nebulus,False,1710808074,Midgardsormr,65
3,False,187,99,Wren Evermist,False,1710806348,Cactuar,79
4,False,200,10,Celestial Tigress,False,1710788946,Midgardsormr,65
...,...,...,...,...,...,...,...,...
32598,False,189,99,Razor Wolfin,False,1667096314,Siren,57
32599,False,150,99,Razor Wolfin,False,1667096312,Siren,57
32600,False,149,90,Razor Wolfin,False,1667096311,Siren,57
32601,False,147,89,Razor Wolfin,False,1667096310,Siren,57


In [25]:
# convert the timestamp to a datetime
entries['timestamp'] = pd.to_datetime(entries['timestamp'], unit='s')
entries

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,False,200,72,Pomo Pamo,False,2024-03-19 00:35:53,Cactuar,79
1,False,187,99,Venase Bosco,False,2024-03-19 00:31:54,Cactuar,79
2,False,205,99,Arzedeu Nebulus,False,2024-03-19 00:27:54,Midgardsormr,65
3,False,187,99,Wren Evermist,False,2024-03-18 23:59:08,Cactuar,79
4,False,200,10,Celestial Tigress,False,2024-03-18 19:09:06,Midgardsormr,65
...,...,...,...,...,...,...,...,...
32598,False,189,99,Razor Wolfin,False,2022-10-30 02:18:34,Siren,57
32599,False,150,99,Razor Wolfin,False,2022-10-30 02:18:32,Siren,57
32600,False,149,90,Razor Wolfin,False,2022-10-30 02:18:31,Siren,57
32601,False,147,89,Razor Wolfin,False,2022-10-30 02:18:30,Siren,57


In [26]:
marketable_consumables_df


,ID,Icon,Name,Url,UrlType,_,_Score
0,4551,/i/020000/020601.png,Potion,/Item/4551,Item,item,0
1,4552,/i/020000/020602.png,Hi-Potion,/Item/4552,Item,item,0
2,4553,/i/020000/020603.png,Mega-Potion,/Item/4553,Item,item,0
3,4554,/i/020000/020603.png,X-Potion,/Item/4554,Item,item,0
4,4555,/i/020000/020621.png,Ether,/Item/4555,Item,item,0
...,...,...,...,...,...,...,...
2316,41408,/i/028000/028116.png,Gharlichthys,/Item/41408,Item,item,0
2317,41409,/i/028000/028562.png,Snowy Parexus,/Item/41409,Item,item,0
2318,41410,/i/028000/028564.png,Furcacauda,/Item/41410,Item,item,0
2319,41411,/i/029000/029780.png,Lopoceras Elegans,/Item/41411,Item,item,0


In [27]:
# now to get the data for all the marketable consumables
# Initialize an empty df to store all results
all_results = pd.DataFrame()

# Initialize the Data center to Aether
# Initialize the histInSeconds to 1 years

world = "Aether"
years = 1
histInSeconds = years*365*24*60*60 # year in seconds

# Loop through all the marketable consumables
# Each request will be for a single consumable
# We have 2321 marketable consumables and we have to limit the requests to 25 per second
# We will use a sleep timer to limit the requests to 25 per second
failed_requests = []

for index, row in marketable_consumables_df.iterrows():
    # Make the request
    itemID = row['ID']
    response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
    # check if the request was successful
    if response.status_code != 200:
        print(f"Request failed for {itemID}")
        # store the itemID in a list to retry later
        failed_requests.append(itemID)
        continue
    response = response.json()
    # normalize the data
    response = pd.json_normalize(response)
    # concatenate the data to the all_results df
    all_results = pd.concat([all_results, response,], ignore_index=True)
    time.sleep(0.04)


Request failed for 4587
Request failed for 4603
Request failed for 7571
Request failed for 21301
Request failed for 36064


In [28]:
#retry the failed requests
for itemID in failed_requests:
    response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
    # check if the request was successful
    if response.status_code != 200:
        print(f"Request failed for {itemID}")
        continue
    response = response.json()
    # normalize the data
    response = pd.json_normalize(response)
    # concatenate the data to the all_results df
    all_results = pd.concat([all_results, response,], ignore_index=True)
    time.sleep(0.04)

In [29]:
# Save the data to a csv file
all_results.to_csv('..\\data\\all_results_1_year.csv', index=False)

In [30]:
#how many items were successfully collected
all_results.shape

(2321, 304)